In [1]:
import pandas as pd
import re

import past_guest_module as pm

import sys
sys.path.append('../zoho_data_app/')

import main_module as m

%load_ext autoreload
%autoreload 2

In [2]:

path = r'C:\Users\fajar.fatoni\Documents\Python\Data\check_country_code_analytic.xlsx'
df_analytic = pd.read_excel(path, dtype='string', sheet_name='Sheet1')
count_data_awal = df_analytic.shape[0]
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KSH024606,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF312348,Mr,Allan,Williams,Male,...,2025-08-03 00:00:00,1,DIR,WEDDING,GROUP,2,0,0,Email and Phone,NO
1,KK076213,Karma Resorts,Other,Past Guests,Karma Kandara,PF319144,Mr.,Aaron Simon,Panario,Male,...,2025-08-07 00:00:00,4,OTA,OTA,RETAIL,3,0,0,Email and Phone,NO
2,KK078068,Karma Resorts,Other,Past Guests,Karma Kandara,PF324380,Mr.,Rashid Ahmed Mohamed Alroma,Alromaithi,Male,...,2025-08-06 00:00:00,5,HOUSE_USE,V_OWNER,COMP,4,2,0,Email and Phone,NO
3,KLM009004,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF310318,Mr,Adam,Brown,Female,...,2025-08-03 00:00:00,2,DIR,WEDDING,GROUP,2,0,0,Email and Phone,NO
4,KSM015289,Karma Resorts,Other,Past Guests,Karma St Martins,PF098974,Mr,Alastair,Mcgregor,Male,...,2025-08-08 00:00:00,7,DIR,TEL,RETAIL,2,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,KK076006,Karma Resorts,Other,Past Guests,Karma Kandara,PF318419,Mr,John,Morrison,Male,...,2025-08-08 00:00:00,7,OTA,OTA,RETAIL,2,1,0,Phone Only,NO
135,KB046167,Karma Resorts,Other,Past Guests,Karma Bavaria,PF295187,<NA>,Anouschka,Hoffmann,Unknown,...,2025-08-03 00:00:00,3,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
136,KB051232,Karma Resorts,Other,Past Guests,Karma Bavaria,PF317751,<NA>,Nadine,Pemp,Unknown,...,2025-08-10 00:00:00,1,OTA,OTA,RETAIL,2,1,0,Phone Only,NO
137,KSM016623,Karma Resorts,Other,Past Guests,Karma St Martins,PF325081,Mr,Steve,Highman,Unknown,...,2025-08-05 00:00:00,1,DIR,TEL,RETAIL,2,1,0,Phone Only,NO


In [3]:
df_analytic.columns

Index(['Booking_Id', 'Lead Brand', 'Lead Sub- Brand', 'Lead Source',
       'Resort_Name', 'ProfileRef', 'Salutation', 'First_Name', 'Surname',
       'Gender', 'Language', 'DOB', 'Street', 'Area', 'Town', 'County',
       'Country', 'Nationality', 'PostCode', 'TelNo', 'MobileNo', 'Email',
       'Exclude From Mailing', 'Opt In', 'Booking Status', 'DateArrival',
       'DateDepart', 'No. of Nights', 'Market_Segment', 'Media_Code',
       'Media Code Group', 'No. of Adult', 'No. of Children', 'No. of Infants',
       'Contact Type', 'Odyssey Members'],
      dtype='object')

In [4]:
df_analytic.fillna('empty', inplace=True)

In [5]:
path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Phone Members.xlsx'
df_phone = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Email Members.xlsx'
df_email = pd.read_excel(path, dtype='string', sheet_name='Sheet1')


In [6]:
m.clean_space(df_email, 'Email')
df_email['Email'] = m.lowercase(df_email, 'Email')

c:\Users\fajar.fatoni\Documents\Python\Task Offline\past_guest_toni_app\../zoho_data_app\main_module.py:127: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[df_columns].replace(' ','', regex=True, inplace=True)


In [7]:
df_analytic.loc[
    (df_analytic['TelNo'] == 'empty') & (df_analytic['MobileNo'] != 'empty'), 
    'TelNo'] = df_analytic['MobileNo']
df_analytic.loc[df_analytic['TelNo'] == df_analytic['MobileNo'],'MobileNo'] = 'empty'

In [8]:
# hitung jumlah duplicate
count_duplicate_general = df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])].shape[0]

df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])]

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
11,KB049834,Karma Resorts,Other,Past Guests,Karma Bavaria,PF310735,Herr,Andreas,Berg,Male,...,2025-08-05 21:00:00,0,CB,CB,GROUP,1,0,0,Email and Phone,NO
15,KB052479,Karma Resorts,Other,Past Guests,Karma Bavaria,PF315209,Frau,Anouschka,Hoffmann,Female,...,2025-08-04 10:30:00,0,COMP,LGR,GROUP,1,0,0,Email and Phone,NO
16,KB050784,Karma Resorts,Other,Past Guests,Karma Bavaria,PF315209,Frau,Anouschka,Hoffmann,Female,...,2025-08-04 11:00:00,0,WEDDING,TEL,RETAIL,1,0,0,Email and Phone,NO
17,KB052252,Karma Resorts,Other,Past Guests,Karma Bavaria,PF315209,Frau,Anouschka,Hoffmann,Female,...,2025-08-03 00:00:00,3,LGR,TEL,RETAIL,2,0,0,Email and Phone,NO
18,KB050783,Karma Resorts,Other,Past Guests,Karma Bavaria,PF315209,Frau,Anouschka,Hoffmann,Female,...,2025-08-03 11:00:00,0,WEDDING,TEL,RETAIL,1,0,0,Email and Phone,NO
26,KLM007648,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF280978,Ms,Caroline,Kennedy,Female,...,2025-08-08 00:00:00,1,WEDDING,WEDDING,GROUP,2,0,0,Email and Phone,NO
73,KSH023844,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF308581,Mr,Mark,Williams,Male,...,2025-08-03 00:00:00,1,DIR,WEDDING,GROUP,2,0,0,Email and Phone,NO
76,KSH025178,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF315941,Ms,Getting Ready,Room,Female,...,2025-08-08 00:00:00,1,DIR,WEDDING,GROUP,2,0,0,Email Only,NO
80,KB039794,Karma Resorts,Other,Past Guests,Karma Bavaria,PF261184,Frau,Katharian,Meiranke,Female,...,2025-08-06 18:00:00,0,CB,CB,GROUP,1,0,0,Email and Phone,NO
81,KB039796,Karma Resorts,Other,Past Guests,Karma Bavaria,PF261184,Frau,Katharian,Meiranke,Female,...,2025-08-08 18:00:00,0,CB,CB,GROUP,1,0,0,Email and Phone,NO


In [9]:
df_analytic.drop_duplicates(subset=['TelNo','MobileNo','Email'], inplace=True)
df_analytic.reset_index(drop=True, inplace=True)

# hitung jumlah setelah value duplicate dihapus
count_after_dedup_duplicate = df_analytic.shape[0]

df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KSH024606,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF312348,Mr,Allan,Williams,Male,...,2025-08-03 00:00:00,1,DIR,WEDDING,GROUP,2,0,0,Email and Phone,NO
1,KK076213,Karma Resorts,Other,Past Guests,Karma Kandara,PF319144,Mr.,Aaron Simon,Panario,Male,...,2025-08-07 00:00:00,4,OTA,OTA,RETAIL,3,0,0,Email and Phone,NO
2,KK078068,Karma Resorts,Other,Past Guests,Karma Kandara,PF324380,Mr.,Rashid Ahmed Mohamed Alroma,Alromaithi,Male,...,2025-08-06 00:00:00,5,HOUSE_USE,V_OWNER,COMP,4,2,0,Email and Phone,NO
3,KLM009004,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF310318,Mr,Adam,Brown,Female,...,2025-08-03 00:00:00,2,DIR,WEDDING,GROUP,2,0,0,Email and Phone,NO
4,KSM015289,Karma Resorts,Other,Past Guests,Karma St Martins,PF098974,Mr,Alastair,Mcgregor,Male,...,2025-08-08 00:00:00,7,DIR,TEL,RETAIL,2,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,KB051232,Karma Resorts,Other,Past Guests,Karma Bavaria,PF317751,empty,Nadine,Pemp,Unknown,...,2025-08-10 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
117,KK076006,Karma Resorts,Other,Past Guests,Karma Kandara,PF318419,Mr,John,Morrison,Male,...,2025-08-08 00:00:00,7,OTA,OTA,RETAIL,2,1,0,Phone Only,NO
118,KB046167,Karma Resorts,Other,Past Guests,Karma Bavaria,PF295187,empty,Anouschka,Hoffmann,Unknown,...,2025-08-03 00:00:00,3,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
119,KSM016623,Karma Resorts,Other,Past Guests,Karma St Martins,PF325081,Mr,Steve,Highman,Unknown,...,2025-08-05 00:00:00,1,DIR,TEL,RETAIL,2,1,0,Phone Only,NO


In [10]:
find_member_booking = df_analytic.copy()

find_member_booking.loc[find_member_booking['TelNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile' 
find_member_booking.loc[find_member_booking['MobileNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile'

find_member_booking.loc[find_member_booking['Email'].isin(df_email['Email']), 'Same Value Find'] = 'Member-Email'
find_member_booking.drop(find_member_booking.loc[find_member_booking['Same Value Find'].isna()].index, inplace=True)

find_member_booking.replace('empty', '', inplace=True)

In [11]:
find_member_booking.to_csv(r'C:\Users\fajar.fatoni\Documents\Python\Data\cek_member_zoho_analytic.csv', index=False)

In [12]:
count_dedup_member = df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])].shape[0]


df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])]


,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
23,KB051161,Karma Resorts,Other,Past Guests,Karma Bavaria,PF147772,Frau,Claudia,Goebl,Female,...,2025-08-09 00:00:00,5,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO
32,KB042786,Karma Resorts,Other,Past Guests,Karma Bavaria,PF276256,Prof.,Dietrich,Grashoff,Unknown,...,2025-08-08 00:00:00,7,COMP,V_OWNER,COMP,4,0,0,Email and Phone,NO
77,KK078276,Karma Resorts,Other,Past Guests,Karma Kandara,PF276011,Mr,Nick,Bowden,Male,...,2025-08-03 00:00:00,1,HOUSE_USE,V_OWNER_NRB,COMP,2,0,0,Email and Phone,NO
79,KSM016609,Karma Resorts,Other,Past Guests,Karma St Martins,PF077988,Mr,Pawel,Wysocki,Male,...,2025-08-07 00:00:00,2,DIR,WALKIN,RETAIL,2,0,0,Email and Phone,NO
88,KB050412,Karma Resorts,Other,Past Guests,Karma Bavaria,PF213986,Ms,Sabine,Reiner,Female,...,2025-08-10 00:00:00,1,COMP,GER_OWE,COMP,1,0,0,Email and Phone,NO
94,KK067223,Karma Resorts,Other,Past Guests,Karma Kandara,PF295577,empty,Sourab,Pramanik,Unknown,...,2025-08-07 00:00:00,6,COMP,RESERVED,KC,1,0,0,Email and Phone,NO


In [13]:
df_analytic.drop(df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])
].index, inplace=True)

df_analytic.reset_index(drop=True, inplace=True)

count_after_dedup_member = df_analytic.shape[0]

In [14]:
count_data_akhir = df_analytic.shape[0]

In [15]:
df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KSH024606,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF312348,Mr,Allan,Williams,Male,...,2025-08-03 00:00:00,1,DIR,WEDDING,GROUP,2,0,0,Email and Phone,NO
1,KB051769,Karma Resorts,Other,Past Guests,Karma Bavaria,PF320837,Mr,Murray,Mckenzie,Male,...,2025-08-03 00:00:00,7,WYNDHAM,WYNDHAM,WYNDHAM,2,0,0,Email and Phone,NO
2,KB051492,Karma Resorts,Other,Past Guests,Karma Bavaria,PF319601,Frau,Monika,Pechloff,Female,...,2025-08-03 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
3,KB052252,Karma Resorts,Other,Past Guests,Karma Bavaria,PF323651,Frau,Melina,Hellmann,Unknown,...,2025-08-03 00:00:00,2,LGR,TEL,RETAIL,6,0,0,Email Only,NO
4,KK077908,Karma Resorts,Other,Past Guests,Karma Kandara,PF323898,Miss,Xiaoling,Ma,Female,...,2025-08-03 00:00:00,3,HOUSE_USE,V_OWNER_NRB,COMP,5,2,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,KSM015828,Karma Resorts,Other,Past Guests,Karma St Martins,PF317077,empty,Erin,Ness,Unknown,...,2025-08-09 00:00:00,5,OTA,OTA,RETAIL,1,1,0,Phone Only,NO
111,KB050923,Karma Resorts,Other,Past Guests,Karma Bavaria,PF316052,empty,Dominik,Hohenleitner,Unknown,...,2025-08-10 00:00:00,1,OTA,OTA,RETAIL,4,0,0,Phone Only,NO
112,KB051232,Karma Resorts,Other,Past Guests,Karma Bavaria,PF317751,empty,Nadine,Pemp,Unknown,...,2025-08-10 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
113,KB044131,Karma Resorts,Other,Past Guests,Karma Bavaria,PF284125,Herr,Wolfgang,Schuller,Male,...,2025-08-11 00:00:00,4,OTA,OTA,RETAIL,2,0,0,Phone Only,NO


In [16]:
df_analytic['DOB'].loc[df_analytic['DOB']=='']

Series([], Name: DOB, dtype: string)

In [17]:
df_analytic['DOB'].replace('empty', '', inplace=True)
dob_con_date = pd.to_datetime(df_analytic['DOB'], errors='coerce')
dob_date = dob_con_date.dt.strftime('%Y-%m-%d')

arr_con_date = pd.to_datetime(df_analytic['DateArrival'])
arr_date = arr_con_date.dt.strftime('%Y-%m-%d')

dep_con_date = pd.to_datetime(df_analytic['DateDepart'])
dep_date = dep_con_date.dt.strftime('%Y-%m-%d')

df_analytic['DOB'] = dob_date
df_analytic['DateArrival'] = arr_date
df_analytic['DateDepart'] = dep_date


C:\Users\fajar.fatoni\AppData\Local\Temp\ipykernel_17464\1513714307.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_analytic['DOB'].replace('empty', '', inplace=True)


In [18]:
df_analytic.replace('empty', '', inplace=True)

df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KSH024606,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF312348,Mr,Allan,Williams,Male,...,2025-08-03,1,DIR,WEDDING,GROUP,2,0,0,Email and Phone,NO
1,KSH024561,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF312044,Mrs,Wendy,Thompson,Female,...,2025-08-03,1,DIR,WEDDING,GROUP,2,0,0,Email and Phone,NO
2,KK078301,Karma Resorts,Other,Past Guests,Karma Kandara,PF324884,Mrs,Ayu,Andiyanti,Female,...,2025-08-03,1,DIR,WALKIN,RETAIL,7,0,0,Email and Phone,NO
3,KSH024577,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF312164,Mr,Simon,Bond,Male,...,2025-08-03,1,DIGITAL,WEDDING,GROUP,2,0,0,Email and Phone,NO
4,KSH024617,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF312445,Mrs,Sally,Nions,Female,...,2025-08-03,1,DIR,WEDDING,GROUP,2,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,KSM015828,Karma Resorts,Other,Past Guests,Karma St Martins,PF317077,,Erin,Ness,Unknown,...,2025-08-09,5,OTA,OTA,RETAIL,1,1,0,Phone Only,NO
111,KB050923,Karma Resorts,Other,Past Guests,Karma Bavaria,PF316052,,Dominik,Hohenleitner,Unknown,...,2025-08-10,1,OTA,OTA,RETAIL,4,0,0,Phone Only,NO
112,KB051232,Karma Resorts,Other,Past Guests,Karma Bavaria,PF317751,,Nadine,Pemp,Unknown,...,2025-08-10,1,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
113,KB044131,Karma Resorts,Other,Past Guests,Karma Bavaria,PF284125,Herr,Wolfgang,Schuller,Male,...,2025-08-11,4,OTA,OTA,RETAIL,2,0,0,Phone Only,NO


In [19]:
print("==========================================================")
print("Summary")
print("==========================================================")
print("Jumlah Data Setelah Script Past Guest Sebelumnya : "+str(count_data_awal))
print("Jumlah Data Yang Duplicate : "+str(count_duplicate_general))
print("Jumlah Data Setelah Hapus Duplicate : "+str(count_after_dedup_duplicate))
print("Jumlah Data Yang Termasuk Member: "+str(count_dedup_member))
print("Jumlah Data Setelah Hapus Member : "+str(count_after_dedup_member))
print("Jumlah Data Akhir : "+str(count_data_akhir))

Summary
Jumlah Data Setelah Script Past Guest Sebelumnya : 139
Jumlah Data Yang Duplicate : 18
Jumlah Data Setelah Hapus Duplicate : 121
Jumlah Data Yang Termasuk Member: 6
Jumlah Data Setelah Hapus Member : 115
Jumlah Data Akhir : 115


In [20]:
df_analytic.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\final_analytics.xlsx', index=False)